In [1]:
import statsmodels.api as sm
from statsmodels.formula.api import ols
from statsmodels.stats.anova import anova_lm
from statsmodels.stats.weightstats import ztest
# from statsmodels.base import elastic_net as els # 这个不会用。。用sklearn做弹性网
from sklearn import linear_model as lm

import seaborn as sbn
import researchpy as rp

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
plt.rcParams['font.sans-serif']=['simhei']
plt.rcParams['axes.unicode_minus']=False

In [10]:
## 定义绘图方法
def draw_freq(df, col_name, save_path="./image/"):
    ## 计数
    y = df[col_name].value_counts()
    x = y.index
    
    ## 绘制柱状图
    plt.figure(figsize=(10,10))
    plt.bar(x,y, width=0.5)
    for a, b in zip(x, y):
        plt.text(a, b + 0.1, '%.0f' % b, ha='center', va='bottom', fontsize=15)
    plt.title("Histogram Frequency Distribution of Variable "+col_name, fontsize=10)
#     plt.legend(handles=[line1, line2, line3, line4, line5], labels=["With","Without","Complete","Traditional", "Non"],fontsize=30)
    plt.yticks(fontsize=10)
    plt.ylabel("Freq",fontsize=10)
    plt.xticks(fontsize=10)
    plt.grid(linestyle="--", alpha=0.3)
    plt.savefig(save_path+col_name+'_bar.png',dpi=600)
    plt.close()
    
    ## 绘制饼图
    plt.figure(figsize=(5,5))
    plt.pie(y,labels=x,autopct='%d%%')
    plt.title("Pie Chart of Frequency Distribution of Variable "+col_name, fontsize=10)
    plt.legend(fontsize=10)
    plt.savefig(save_path+col_name+'_pie.png',dpi=600)
    plt.close()

def draw_hist(df, col_name, save_path="./image/"):
    # 首先计算极差，根据极差改柱体数量。本实验只有缺勤指标较大，因此直接输出就行
    col_range = df[col_name].max()-df[col_name].min()
    bins = col_range+1
    ## 直方图
    fig,ax = plt.subplots()
    n,bins_num,pat = ax.hist(df[col_name], bins, (df[col_name].min()-0.5, df[col_name].max()+0.5))
    ax.plot([i+0.5 for i in bins_num[:-1]], n)
    
    plt.title("Histogram Frequency Distribution of Variable "+col_name, fontsize=10)
    plt.ylabel("Freq",fontsize=10)
    plt.xlabel(col_name,fontsize=10)
    plt.savefig(save_path+col_name+'_hist.png',dpi=600)
    plt.close()

In [2]:
## 读入数据
df_raw = pd.read_csv("./data/student-mat.csv", sep=";", header=0, encoding='UTF-8')

In [3]:
## 检查数据
### 检查是否有缺失值
print("数据中缺失值的个数为：\n", df_raw.isnull().sum())

数据中缺失值的个数为：
 school        0
sex           0
age           0
address       0
famsize       0
Pstatus       0
Medu          0
Fedu          0
Mjob          0
Fjob          0
reason        0
guardian      0
traveltime    0
studytime     0
failures      0
schoolsup     0
famsup        0
paid          0
activities    0
nursery       0
higher        0
internet      0
romantic      0
famrel        0
freetime      0
goout         0
Dalc          0
Walc          0
health        0
absences      0
G1            0
G2            0
G3            0
dtype: int64


In [8]:
### 实验报告废话生成：
def feihua_out_1(df):
    
    file = open('./out.txt','w')
    
    for i in range(len(df)):
        str_initial = "学生 ({})的统计量的极差为{:.0f}，范围为{:.0f}-{:.0f}，中位数为{:.0f}，均值（方差，标准差）为{:.2f}({:.2f}, {:.2f})，峰度(Kurt)为{:.2f}，偏度(Skew)为{:.2f}。".format(df["Variable"][i],df["Range"][i],df["min"][i],df["max"][i],df["50%"][i],df["Mean"][i],df["SD"][i],df["SE"][i],df["Kurtosis"][i],df["Skew"][i])
        
        str_0_0 = "峰度小于0，说明数据较为分散；偏度小于0，说明数据整体呈左偏分布。"
        str_0_1 = "峰度小于0，说明数据较为分散；偏度大于0，说明数据整体呈右偏分布。"
        str_1_0 = "峰度大于0，说明数据较为集中；偏度小于0，说明数据整体呈左偏分布。"
        str_1_1 = "峰度大于0，说明数据较为集中；偏度大于0，说明数据整体呈右偏分布。"
        str_out = ""
        if df["Kurtosis"][i]<0:
            if df["Skew"][i]<0:
                str_out = str_initial + str_0_0
            else:
                str_out = str_initial + str_0_1
        else:
            if df["Skew"][i]<0:
                str_out = str_initial + str_1_0
            else:
                str_out = str_initial + str_1_1
        print(str_out,file=file)
    
    file.close()

def feihua_out_2(df,col_name):
    file = open('./out2.txt','a')
    class_list = list(df.index)[:-1]
    n_classes = len(class_list)
    min_name = ""
    max_name = ""
    min_freq = 395
    max_freq = 0
    
    str_initial = "学生{}的统计量为名义变量，共有{:.0f}类，其中".format(col_name, n_classes)
    for i in range(n_classes):
        if df[col_name][i] >= max_freq:
            max_freq = df[col_name][i]
            max_name = class_list[i]
        if df[col_name][i] <= min_freq:
            min_freq = df[col_name][i]
            min_name = class_list[i]
        add_str = "类别({})的频数为{:.0f}，占比{:.2%},".format(class_list[i], df[col_name][i], df["proportion(%)"][i]/100)
        str_initial += add_str
    str_end = "由此可见，该变量中，类别为{}的学生人数占比最多(频数为{:.0f})，类别为{}的学生人数占比最少(频数为{:.0f})。" .format(max_name, max_freq, min_name, min_freq)
    str_out = str_initial + str_end
    print(str_out,file=file)
    file.close()

def feihua_out_3(df, col_name):
    
    file = open('./out3.txt','a')
    
    for i in range(len(df)):
        str_initial = "学生 ({})分类为{}的人数为{:.0f}，统计量的极差为{:.0f}，范围为{:.0f}-{:.0f}，中位数为{:.0f}，均值（标准差）为{:.2f}({:.2f})，峰度(Kurt)为{:.2f}，偏度(Skew)为{:.2f}。".format(col_name,df[col_name][i],df["count"][i],df["Range"][i],df["min"][i],df["max"][i],df["50%"][i],df["mean"][i],df["std"][i],df["Kurtosis"][i],df["Skew"][i])
        
        str_0_0 = "峰度小于0，说明数据较为分散；偏度小于0，说明数据整体呈左偏分布。"
        str_0_1 = "峰度小于0，说明数据较为分散；偏度大于0，说明数据整体呈右偏分布。"
        str_1_0 = "峰度大于0，说明数据较为集中；偏度小于0，说明数据整体呈左偏分布。"
        str_1_1 = "峰度大于0，说明数据较为集中；偏度大于0，说明数据整体呈右偏分布。"
        str_out = ""
        if df["Kurtosis"][i]<0:
            if df["Skew"][i]<0:
                str_out = str_initial + str_0_0
            else:
                str_out = str_initial + str_0_1
        else:
            if df["Skew"][i]<0:
                str_out = str_initial + str_1_0
            else:
                str_out = str_initial + str_1_1
        print(str_out,file=file)
    
    file.close()

def feihua_out_4(df, col_name):
    file = open('./out4.txt','a')
    
    str_out = "给定原假设H_0为：不同{}下因变量无差异；备择假设为：不同{}下因变量有差异。\n经过方差分析检验(ANOVA)，在给定显著水平 {}=0.05的情况下，p值为{}，".format(col_name,col_name,chr(945),df["PR(>F)"][0])
    str_0 = " 大于{}，表示在原假设为真的情况下，统计量发生的概率较高，因此接受原假设，说明{}与因变量的相关性不显著。".format(chr(945),col_name)
    str_1 = " 大于{}，表示在原假设为真的情况下，统计量发生的概率较低，因此拒绝原假设，说明{}与因变量的相关性较显著。".format(chr(945),col_name)
    if df["PR(>F)"][0]>=0.05:
        str_out += str_0
    else:
        str_out += str_1
    print(str_out,file=file)
    
    file.close()

In [4]:
## 描述性统计
### 数值型（包括数值变量，数值型分类变量）
col_names_numeric = list(df_raw.describe().columns)
print("数值变量包括：",col_names_numeric)
df_basic_describe = df_raw[col_names_numeric].describe().T

df_describe_numeric = rp.summary_cont(df_raw[col_names_numeric])
#### 偏度和峰度计算
list_kurtosis, list_skew = [],[]
for col_name in col_names_numeric:
    list_kurtosis.append(df_raw[col_name].kurtosis())
    list_skew.append(df_raw[col_name].skew())
#     draw_hist(df_raw,col_name)
df_describe_numeric["Kurtosis"],df_describe_numeric["Skew"]=list_kurtosis, list_skew
df_describe_numeric.index = pd.Series(col_names_numeric)
df_describe_numeric = pd.concat([df_basic_describe[["min","25%","50%","75%","max"]],df_describe_numeric],axis=1)
df_describe_numeric["Range"] = df_basic_describe["max"]-df_basic_describe["min"]
df_describe_numeric.to_excel("./file/describe.xlsx")

### 名义变量
col_names_nominal = [i for i in list(df_raw.columns) if i not in col_names_numeric]
print("名义变量包括：",col_names_nominal)
writer = pd.ExcelWriter("./file/value_counts.xlsx")
for col_name in col_names_nominal:
    df_value_count = pd.DataFrame(df_raw[col_name].value_counts())
    total_freq = df_value_count[col_name].sum()
    df_value_count["proportion(%)"] = df_value_count[col_name]/total_freq*100
    df_value_count.loc["Total"]=[total_freq,100]
    df_value_count.to_excel(writer,sheet_name=col_name)
#     draw_freq(df_raw, col_name)
#     feihua_out_2(df_value_count,col_name)
writer.save()
writer.close()

数值变量包括： ['age', 'Medu', 'Fedu', 'traveltime', 'studytime', 'failures', 'famrel', 'freetime', 'goout', 'Dalc', 'Walc', 'health', 'absences', 'G1', 'G2', 'G3']


名义变量包括： ['school', 'sex', 'address', 'famsize', 'Pstatus', 'Mjob', 'Fjob', 'reason', 'guardian', 'schoolsup', 'famsup', 'paid', 'activities', 'nursery', 'higher', 'internet', 'romantic']


In [9]:
# feihua_out_1(df_describe_numeric)

In [31]:
## 定义分组绘图的方法
def draw_group_by_nominal(df, IV_name, DV_name, save_path="./image/"):
    plt.figure(figsize=(10,10))
    df_each = df[[IV_name, DV_name]]
    fig = sbn.boxplot(x=IV_name, y=DV_name, data=df_each)
    plt.title("Boxplot of Variable "+DV_name+" group by "+IV_name, fontsize=20)
    plt.yticks(fontsize=20)
    plt.ylabel(DV_name,fontsize=20)
    plt.xticks(fontsize=20)
    plt.xlabel(IV_name,fontsize=20)
    plt.grid(linestyle="--", alpha=0.3)
    plt.savefig(save_path+IV_name+'_'+DV_name+'_box_plot.png',dpi=600)
    plt.close()

def draw_group_by_numeric(df, IV_name, DV_name, save_path="./image/"):
    plt.figure(figsize=(10,10))
    df_each = df[[IV_name, DV_name]]
    fig = (sbn.jointplot(x=IV_name, y=DV_name, data=df_each, marginal_ticks=True, kind='reg',color="orange", scatter_kws={'s': [5]}).plot_joint(sbn.kdeplot, color='royalblue'))
#     plt.title("Plot of Variable "+DV_name+" and "+IV_name, fontsize=20)
    plt.yticks(fontsize=20)
    plt.ylabel(DV_name,fontsize=20)
    plt.xticks(fontsize=20)
    plt.xlabel(IV_name,fontsize=20)
    plt.grid(linestyle="--", alpha=0.3)
    fig.savefig(save_path+IV_name+'_'+DV_name+'_scatter_plot.png',dpi=600)
    plt.close()

def draw_heat_map(df, save_path="./image/"):
    ## this df is the cor
    plt.figure(figsize=(12,12))
    fig = sbn.heatmap(data = df_numeric.corr(),annot = True, fmt=".2f", vmin=-1.0, vmax=1.0, cmap="RdBu_r")
    fig.set(xlabel="", ylabel="")
    fig.xaxis.tick_top()
    plt.title("Plot of Cor", fontsize=20)
    plt.savefig(save_path+'heat_map_cor.png',dpi=600)
    plt.close()

In [96]:
def my_anova(df, DV_name, IV_name, writer):
    
    df_each = df[[IV_name,DV_name]]
    model_each = ols('G3~C('+IV_name+')', data=df_each).fit()
    anova_test = anova_lm(model_each)
    anova_test.to_excel(writer,sheet_name=IV_name)
    
    return anova_test

In [99]:
## 分组的因变量描述性统计 （因变量为G3，因此只对G3做分组描述统计）
### 分类变量与G3
writer = pd.ExcelWriter("./file/groupby_describe.xlsx")
for col_name in col_names_nominal:
    df_each = df_raw[[col_name,'G3']].groupby([col_name]).describe()
    df_each = df_each.droplevel(0,axis=1)
    df_each.index.name=""   
    df_each.insert(loc=0, column=col_name, value=list(df_each.index))
    
    df_each["Skew"] = list(df_raw[[col_name,'G3']].groupby([col_name]).skew()["G3"])
    df_each["Kurtosis"] = list(df_raw[[col_name,'G3']].groupby([col_name]).apply(pd.DataFrame.kurt)["G3"])
    df_each["Range"] = df_each[("max")] - df_each[("min")]
    df_each.to_excel(writer,sheet_name="G_by_"+col_name, index=False)

    feihua_out_3(df_each, col_name)
#     draw_group_by_nominal(df_raw, col_name, "G3")

writer.save()
writer.close()

writer = pd.ExcelWriter("./file/anova_nominal.xlsx")
for col_name in col_names_nominal:
    df_anova = my_anova(df_raw, "G3", col_name, writer)
    feihua_out_4(df_anova, col_name)

writer.save()
writer.close()

### 数值变量与G3
df_numeric = df_raw[col_names_numeric]
#### 相关系数检验
df_numeric.corr().to_excel("./file/correlation.xlsx")
#### 相关系数热力图
draw_heat_map(df_numeric.corr())
#### 相关性绝对值高低判断
print(" 与G3正相关的有：\n", ",".join(list(df_numeric.corr()["G3"].loc[df_numeric.corr()["G3"]>=0].index)))
print(" 与G3负相关的有：\n", ",".join(list(df_numeric.corr()["G3"].loc[df_numeric.corr()["G3"]<0].index)))
#### 相关性高低
# df_numeric.corr()["G3"].apply(abs).sort_values(ascending=False).to_excel("./file/cor_rank.xlsx")


#### 挨个画图 会报错，但是能成功保存图片，具体原因未知。
# for col_name in col_names_numeric:
#     draw_group_by_numeric(df_raw, col_name, "G3")

#### 绘制图 已绘制
# sbn.set_theme(style="ticks")

# fig = sbn.pairplot(df_numeric,kind="reg",diag_kind="kde")
# fig.savefig("./image/pairplot_total_1.png", dpi = 600)

# fig = sbn.pairplot(df_numeric,kind="reg",diag_kind="hist")
# fig.savefig("./image/pairplot_total_2.png", dpi = 600)

# fig = sbn.pairplot(df_numeric,x_vars=col_names_numeric, y_vars=["G3"], kind="reg",diag_kind="hist")
# fig.savefig("./image/pairplot_G3.png", dpi = 600)


C:\Users\DELL\AppData\Local\Temp\ipykernel_11268\1187730233.py:11: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  df_each["Kurtosis"] = list(df_raw[[col_name,'G3']].groupby([col_name]).apply(pd.DataFrame.kurt)["G3"])
C:\Users\DELL\AppData\Local\Temp\ipykernel_11268\1187730233.py:11: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  df_each["Kurtosis"] = list(df_raw[[col_name,'G3']].groupby([col_name]).apply(pd.DataFrame.kurt)["G3"])
C:\Users\DELL\AppData\Local\Temp\ipykernel_11268\1187730233.py:11: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Sel

 与G3正相关的有：
 Medu,Fedu,studytime,famrel,freetime,absences,G1,G2,G3
 与G3负相关的有：
 age,traveltime,failures,goout,Dalc,Walc,health


In [11]:
def step_OLS(df,IV_names,DV_name, with_const = False, random_del = False, random_seed = None):
    np.random.seed(random_seed)
    
    new_IV_names = IV_names.copy()
    if with_const:
        LR_full = sm.OLS(df[DV_names], sm.add_constant(df[IV_names])).fit()
    else: 
        LR_full = sm.OLS(df[DV_names], df[IV_names]).fit()
    
    df_pvalues = LR_full.pvalues
    del_variable_list = list(df_pvalues.loc[df_pvalues>0.05].index)
    if 'const' in del_variable_list:
        del_variable_list.remove('const')
    
    if random_del:
        np.random.shuffle(del_variable_list)
    
    LR_step_list = [LR_full]
    while True:
        if len(del_variable_list)>0:
            new_IV_names.remove(del_variable_list[0])
        else:
            break
        LR_step = sm.OLS(df[DV_names], df[new_IV_names]).fit()
        
        df_pvalues = LR_step.pvalues
        del_variable_list = list(df_pvalues.loc[df_pvalues>0.05].index)
        if 'const' in del_variable_list:
            del_variable_list.remove('const')
        if random_del:
            np.random.shuffle(del_variable_list)
        
        LR_step_list.append(LR_step)
    
    return LR_step_list

def step_OLS_to_excel(df, IV_names, DV_name, file_path, with_const = False, random_del = False, random_seed = None):
    LR_step_list = step_OLS(df, IV_names, DV_name, with_const, random_del, random_seed)
    writer = pd.ExcelWriter(file_path)
    for step, i_summary in enumerate([i.summary() for i in LR_step_list]):
        df_table_1 =  pd.read_html(i_summary.tables[0].as_html())[0]

        df_table_2 =  pd.read_html(i_summary.tables[1].as_html())[0]
        df_table_2.columns = np.array(df_table_2).tolist()[0]
        df_table_2.drop([0], inplace=True)
        df_table_2 = df_table_2.reset_index(drop=True)

        df_table_3 =  pd.read_html(i_summary.tables[2].as_html())[0]
        pd.concat([df_table_1, df_table_2, df_table_3], axis=1).to_excel(writer,sheet_name="Step_"+str(step), index=False) 

    writer.save()
    writer.close()
    return LR_step_list


In [14]:
## 多元线性回归
### 使用one-hot对分类变量做处理
df_produce = df_raw.copy()
df_produce = df_produce.drop(columns=["G1","G2"])

for col_name in col_names_nominal:
    df_each = pd.get_dummies(df_produce[col_name],prefix=col_name,drop_first=True)
#     df_each.columns = [col_name+"."+i for i in list(df_each.columns)] 等价于prefix
    df_produce = df_produce.drop(columns=[col_name])
    df_produce = pd.concat([df_produce,df_each],axis=1)
df_produce.to_excel("./data/data_produce.xlsx", index=False)
### 全部变量一起做回归
IV_names = list(df_produce.columns)
IV_names.remove("G3")
DV_names = "G3"

## 带截距项
LR_step_list = step_OLS_to_excel(df_produce,IV_names,DV_names,"./file/OLS_by_step_sequence_const.xlsx", with_const=True, random_del=False)


## 不带截距项

### step regression
#### random_del=False 默认表示按顺序删变量，否则随机删除变量。random_seed表示numpy随机数种子，默认值为None
LR_step_list = step_OLS_to_excel(df_produce,IV_names,DV_names,"./file/OLS_by_step_sequence.xlsx", random_del=False)
# LR_step_list = step_OLS_to_excel(df_produce,IV_names,DV_names,"./file/OLS_by_step_seed.xlsx", random_del=True)
# LR_step_list = step_OLS_to_excel(df_produce,IV_names,DV_names,"./file/OLS_by_step_seed_0.xlsx", random_del=True, random_seed=0)
# LR_step_list = step_OLS_to_excel(df_produce,IV_names,DV_names,"./file/OLS_by_step_seed_1.xlsx", random_del=True, random_seed=1)
# LR_step_list = step_OLS_to_excel(df_produce,IV_names,DV_names,"./file/OLS_by_step_seed_2.xlsx", random_del=True, random_seed=2)
# LR_step_list = step_OLS_to_excel(df_produce,IV_names,DV_names,"./file/OLS_by_step_seed_3.xlsx", random_del=True, random_seed=3)
# LR_step_list = step_OLS_to_excel(df_produce,IV_names,DV_names,"./file/OLS_by_step_seed_4.xlsx", random_del=True, random_seed=4)

In [102]:
### full regression
print(LR_step_list[0].summary())

                                 OLS Regression Results                                
Dep. Variable:                     G3   R-squared (uncentered):                   0.880
Model:                            OLS   Adj. R-squared (uncentered):              0.866
Method:                 Least Squares   F-statistic:                              66.65
Date:                Sat, 29 Oct 2022   Prob (F-statistic):                   9.48e-140
Time:                        18:30:31   Log-Likelihood:                         -1102.9
No. Observations:                 395   AIC:                                      2284.
Df Residuals:                     356   BIC:                                      2439.
Df Model:                          39                                                  
Covariance Type:            nonrobust                                                  
                        coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------

In [15]:
### final regression
print(LR_step_list[-1].summary())

                                 OLS Regression Results                                
Dep. Variable:                     G3   R-squared (uncentered):                   0.858
Model:                            OLS   Adj. R-squared (uncentered):              0.856
Method:                 Least Squares   F-statistic:                              335.8
Date:                Sun, 30 Oct 2022   Prob (F-statistic):                   2.43e-160
Time:                        10:37:40   Log-Likelihood:                         -1134.9
No. Observations:                 395   AIC:                                      2284.
Df Residuals:                     388   BIC:                                      2312.
Df Model:                           7                                                  
Covariance Type:            nonrobust                                                  
                   coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------

In [23]:
## a question: different sequence may have different result. Use np.random.shuffle(<list>) to random the col_name
### 结果表明先后顺序会影响，因此最终结果是个局部最优

### l1_ratio表示l1正则（r）和l2（1-r）正则的程度
elastic_model = lm.ElasticNet(l1_ratio = 1)
elastic_result = elastic_model.fit(df_produce[IV_names], df_produce[DV_names])
df_elastic_result = pd.DataFrame()
df_elastic_result['Variable'] = IV_names
df_elastic_result['coef'] = elastic_result.coef_
df_elastic_result.to_excel('./file/elastic_ls_1.xlsx')

In [7]:
from statsmodels.api import OLS
from statsmodels.formula.api import ols

In [9]:
test_OLS = OLS(df_produce[DV_names],sm.add_constant(df_produce[IV_names])).fit()
print(test_OLS.summary())

                            OLS Regression Results                            
Dep. Variable:                     G3   R-squared:                       0.276
Model:                            OLS   Adj. R-squared:                  0.196
Method:                 Least Squares   F-statistic:                     3.463
Date:                Sun, 30 Oct 2022   Prob (F-statistic):           3.32e-10
Time:                        16:04:27   Log-Likelihood:                -1097.5
No. Observations:                 395   AIC:                             2275.
Df Residuals:                     355   BIC:                             2434.
Df Model:                          39                                         
Covariance Type:            nonrobust                                         
                        coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------
const                14.0777      4.48

In [38]:
test_OLS = OLS(df_produce[DV_names],pd.concat([df_produce[IV_names],pd.DataFrame([1]*len(df_produce))],axis=1)).fit()
print(test_OLS.summary())
test_ols = ols('G3~'+'+'.join(IV_names), data=df_produce).fit()
print(test_ols.summary())

                            OLS Regression Results                            
Dep. Variable:                     G3   R-squared:                       0.276
Model:                            OLS   Adj. R-squared:                  0.196
Method:                 Least Squares   F-statistic:                     3.463
Date:                Sun, 30 Oct 2022   Prob (F-statistic):           3.32e-10
Time:                        11:09:40   Log-Likelihood:                -1097.5
No. Observations:                 395   AIC:                             2275.
Df Residuals:                     355   BIC:                             2434.
Df Model:                          39                                         
Covariance Type:            nonrobust                                         
                        coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------
age                  -0.3752      0.21

In [4]:
help(OLS)

Help on class OLS in module statsmodels.regression.linear_model:

class OLS(WLS)
 |  OLS(endog, exog=None, missing='none', hasconst=None, **kwargs)
 |  
 |  Ordinary Least Squares
 |  
 |  Parameters
 |  ----------
 |  endog : array_like
 |      A 1-d endogenous response variable. The dependent variable.
 |  exog : array_like
 |      A nobs x k array where `nobs` is the number of observations and `k`
 |      is the number of regressors. An intercept is not included by default
 |      and should be added by the user. See
 |      :func:`statsmodels.tools.add_constant`.
 |  missing : str
 |      Available options are 'none', 'drop', and 'raise'. If 'none', no nan
 |      checking is done. If 'drop', any observations with nans are dropped.
 |      If 'raise', an error is raised. Default is 'none'.
 |  hasconst : None or bool
 |      Indicates whether the RHS includes a user-supplied constant. If True,
 |      a constant is not checked for and k_constant is set to 1 and all
 |      result s

In [5]:
help(ols)

Help on method from_formula in module statsmodels.base.model:

from_formula(formula, data, subset=None, drop_cols=None, *args, **kwargs) method of builtins.type instance
    Create a Model from a formula and dataframe.
    
    Parameters
    ----------
    formula : str or generic Formula object
        The formula specifying the model.
    data : array_like
        The data for the model. See Notes.
    subset : array_like
        An array-like object of booleans, integers, or index values that
        indicate the subset of df to use in the model. Assumes df is a
        `pandas.DataFrame`.
    drop_cols : array_like
        Columns to drop from the design matrix.  Cannot be used to
        drop terms involving categoricals.
    *args
        Additional positional argument that are passed to the model.
    **kwargs
        These are passed to the model with one exception. The
        ``eval_env`` keyword is passed to patsy. It can be either a
        :class:`patsy:patsy.EvalEnvironm